# Sample for KFServing SDK 

This is a sample for KFServing SDK. 

The notebook shows how to use KFServing SDK to create, get, patch and delete KFService.

In [17]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2KFServiceSpec
from kfserving import V1alpha2KFService

## Define KFService

Firstly define the model spec for default model spec, and then define the kfservice basic on the model spec.

In [18]:
default_endpoint_spec = V1alpha2EndpointSpec(predictor=V1alpha2PredictorSpec(tensorflow=V1alpha2TensorflowSpec(
                                             storage_uri='gs://kfserving-samples/models/tensorflow/flowers')))
    
kfsvc = V1alpha2KFService(api_version=constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(name='flower-sample', namespace='kubeflow'),
                          spec=V1alpha2KFServiceSpec(default=default_endpoint_spec))

## Create KFService

Call KFServingClient to create KFService.

In [19]:
KFServing = KFServingClient()
KFServing.create(kfsvc)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'kind': 'KFService',
 'metadata': {'creationTimestamp': '2019-09-20T02:35:00Z',
  'generation': 1,
  'name': 'flower-sample',
  'namespace': 'kubeflow',
  'resourceVersion': '894583',
  'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/kubeflow/kfservices/flower-sample',
  'uid': '3e3091c7-db4f-11e9-adc5-42010a80027c'},
 'spec': {'default': {'predictor': {'tensorflow': {'resources': {'limits': {'cpu': '1',
       'memory': '2Gi'},
      'requests': {'cpu': '1', 'memory': '2Gi'}},
     'runtimeVersion': 'latest',
     'storageUri': 'gs://kfserving-samples/models/tensorflow/flowers'}}}},
 'status': {'canary': {}, 'default': {}}}

## Check the KFService

In [20]:
KFServing.get('flower-sample', namespace='kubeflow')

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'kind': 'KFService',
 'metadata': {'creationTimestamp': '2019-09-20T02:35:00Z',
  'generation': 2,
  'name': 'flower-sample',
  'namespace': 'kubeflow',
  'resourceVersion': '894642',
  'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/kubeflow/kfservices/flower-sample',
  'uid': '3e3091c7-db4f-11e9-adc5-42010a80027c'},
 'spec': {'default': {'predictor': {'tensorflow': {'resources': {'limits': {'cpu': '1',
       'memory': '2Gi'},
      'requests': {'cpu': '1', 'memory': '2Gi'}},
     'runtimeVersion': 'latest',
     'storageUri': 'gs://kfserving-samples/models/tensorflow/flowers'}}}},
 'status': {'canary': {},
  'conditions': [{'lastTransitionTime': '2019-09-20T02:35:01Z',
    'message': 'Configuration "flower-sample-predictor-default" does not have any ready Revision.',
    'reason': 'RevisionMissing',
    'status': 'False',
    'type': 'DefaultPredictorReady'},
   {'lastTransitionTime': '2019-09-20T02:35:01Z',
    'message':

## Patch the KFService 

Patch the KFService to add canary model.

In [21]:
canary_endpoint_spec = V1alpha2EndpointSpec(predictor=V1alpha2PredictorSpec(tensorflow=V1alpha2TensorflowSpec(
                                            storage_uri='gs://kfserving-samples/models/tensorflow/flowers')))

kfsvc = V1alpha2KFService(api_version=constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(name='flower-sample', namespace='kubeflow'),
                          spec=V1alpha2KFServiceSpec(default=default_endpoint_spec,
                                                     canary=canary_endpoint_spec,
                                                     canary_traffic_percent=10))

KFServing.patch('flower-sample', kfsvc)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'kind': 'KFService',
 'metadata': {'creationTimestamp': '2019-09-20T02:35:00Z',
  'generation': 3,
  'name': 'flower-sample',
  'namespace': 'kubeflow',
  'resourceVersion': '894693',
  'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/kubeflow/kfservices/flower-sample',
  'uid': '3e3091c7-db4f-11e9-adc5-42010a80027c'},
 'spec': {'canary': {'predictor': {'tensorflow': {'resources': {'limits': {'cpu': '1',
       'memory': '2Gi'},
      'requests': {'cpu': '1', 'memory': '2Gi'}},
     'runtimeVersion': 'latest',
     'storageUri': 'gs://kfserving-samples/models/tensorflow/flowers'}}},
  'canaryTrafficPercent': 10,
  'default': {'predictor': {'tensorflow': {'resources': {'limits': {'cpu': '1',
       'memory': '2Gi'},
      'requests': {'cpu': '1', 'memory': '2Gi'}},
     'runtimeVersion': 'latest',
     'storageUri': 'gs://kfserving-samples/models/tensorflow/flowers'}}}},
 'status': {'canary': {},
  'conditions': [{'lastTransiti

## Delete the KFService

In [ ]:
KFServing.delete('flower-sample', namespace='kubeflow')
